## Преобразуем файлик с движением.

In [1]:
import pandas as pd
import itertools

pd.set_option("future.no_silent_downcasting", True)

In [2]:
df = pd.read_excel(
    r"data\raw\turnover_raw.xlsx"
)

df.head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,Движение оборудования за период,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Параметры:,NaN,Начало периода: 01.01.2016 0:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,Конец периода: 07.10.2024 23:59:59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Отбор:,NaN,"Склад В списке ""Склад Уткина Заводь (обор...; ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Номенклатура.Код,NaN,NaN,Период.Начало дня,Регистратор,NaN,NaN,Регистратор.Склад отправитель,Регистратор.Склад получатель,Приход,Расход,Конечное сальдо
7,643220752,NaN,NaN,16.01.2016,Приходная накладная 417 от 16.01.2016 0:00:00,NaN,NaN,NaN,NaN,110,NaN,110
8,643153277,NaN,NaN,16.01.2016,Приходная накладная 417 от 16.01.2016 0:00:00,NaN,NaN,NaN,NaN,30,NaN,30
9,643193171,NaN,NaN,16.01.2016,Приходная накладная 417 от 16.01.2016 0:00:00,NaN,NaN,NaN,NaN,362,NaN,362


In [31]:
cols = df.iloc[6]

cols

Unnamed: 0                             Склад
Unnamed: 1                               NaN
Unnamed: 2                               NaN
Unnamed: 3                  Номенклатура.Код
Unnamed: 4                 Период.Начало дня
Unnamed: 5                               NaN
Unnamed: 6                       Регистратор
Unnamed: 7                               NaN
Unnamed: 8     Регистратор.Склад отправитель
Unnamed: 9      Регистратор.Склад получатель
Unnamed: 10                           Приход
Unnamed: 11                           Расход
Unnamed: 12                  Конечное сальдо
Name: 6, dtype: object

In [32]:
data = df[7:].reset_index(drop=True)
data.columns = cols
data.dropna(axis=1, how="all", inplace=True)
data = data[data["Склад"] != "Итого"]
data.head()

6,Склад,Номенклатура.Код,Период.Начало дня,Регистратор,Регистратор.Склад отправитель,Регистратор.Склад получатель,Приход,Расход,Конечное сальдо
0,Склад Оборудования в СПб,829754,29.01.2019,Приходная накладная 762 от 29.01.2019 16:19:21,NaN,NaN,1,NaN,1
1,Склад Оборудования в СПб,829754,30.01.2019,Перемещение оборудования в рознице 87 061 от 3...,Склад Оборудования в СПб,Июнь LR (Череповец),NaN,1,NaN
2,Склад Оборудования в СПб,833325,16.01.2019,Приходная накладная 417 от 16.01.2019 15:47:03,NaN,NaN,1,NaN,1
3,Склад Оборудования в СПб,833326,16.01.2019,Приходная накладная 417 от 16.01.2019 15:47:03,NaN,NaN,1,NaN,1
4,Склад Оборудования в СПб,833327,16.01.2019,Приходная накладная 417 от 16.01.2019 15:47:03,NaN,NaN,1,NaN,1


In [33]:
data.dropna(axis=1, how="all", inplace=True)
data.columns = [
    "warehouse_id",
    "unit_id",
    "date",
    "document",
    "sender",
    "recipient",
    "income",
    "outcome",
    "end_balance",
]

# Уберем отрицательные значения
data.end_balance = data.end_balance.apply(lambda x: 0 if x < 0 else x)

# Преобразуем типы
data["date"] = pd.to_datetime(data["date"], dayfirst=True)

# Заполним пропуски
columns = ["document", "sender", "recipient", "income", "outcome", "end_balance"]
for col in columns:
    data[col] = data[col].fillna(0)
data["income"] = data["income"].astype(float)
data["outcome"] = data["outcome"].astype(float)

data.head()


,warehouse_id,unit_id,date,document,sender,recipient,income,outcome,end_balance
0,Склад Оборудования в СПб,829754,2019-01-29,Приходная накладная 762 от 29.01.2019 16:19:21,0,0,1.0,0.0,1.0
1,Склад Оборудования в СПб,829754,2019-01-30,Перемещение оборудования в рознице 87 061 от 3...,Склад Оборудования в СПб,Июнь LR (Череповец),0.0,1.0,0.0
2,Склад Оборудования в СПб,833325,2019-01-16,Приходная накладная 417 от 16.01.2019 15:47:03,0,0,1.0,0.0,1.0
3,Склад Оборудования в СПб,833326,2019-01-16,Приходная накладная 417 от 16.01.2019 15:47:03,0,0,1.0,0.0,1.0
4,Склад Оборудования в СПб,833327,2019-01-16,Приходная накладная 417 от 16.01.2019 15:47:03,0,0,1.0,0.0,1.0


In [34]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 491542 entries, 0 to 491541
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   warehouse_id  491542 non-null  object        
 1   unit_id       491542 non-null  object        
 2   date          491542 non-null  datetime64[ns]
 3   document      491542 non-null  object        
 4   sender        491542 non-null  object        
 5   recipient     491542 non-null  object        
 6   income        491542 non-null  float64       
 7   outcome       491542 non-null  float64       
 8   end_balance   491542 non-null  float64       
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 37.5+ MB


В данных движения имеются склады, движение между которыми не считается отгрузкой. Добавим признак, который будет помечать такие отгрузки для обнуления будущей метрики.

In [35]:
def is_valuable_send(row, warehouses):
    for i, j in itertools.combinations(warehouses, 2):
        if (
            ((row.sender == i) & (row.recipient == j))
            or ((row.sender == j) & (row.recipient == i))
            or "Корректировка" in str(row.document)
        ):
            return 1
    return 0


warehouses = [
    "Форстранс-Кубинская",
    "Сервис Логистика Спб ",
    "Склад Уткина Заводь (оборудование)",
    "Склад Оборудования в СПб",
]


data["not_valuable_send"] = data.apply(
    lambda row: is_valuable_send(row, warehouses), axis=1
)

data.head()

,warehouse_id,unit_id,date,document,sender,recipient,income,outcome,end_balance,not_valuable_send
0,Склад Оборудования в СПб,829754,2019-01-29,Приходная накладная 762 от 29.01.2019 16:19:21,0,0,1.0,0.0,1.0,0
1,Склад Оборудования в СПб,829754,2019-01-30,Перемещение оборудования в рознице 87 061 от 3...,Склад Оборудования в СПб,Июнь LR (Череповец),0.0,1.0,0.0,0
2,Склад Оборудования в СПб,833325,2019-01-16,Приходная накладная 417 от 16.01.2019 15:47:03,0,0,1.0,0.0,1.0,0
3,Склад Оборудования в СПб,833326,2019-01-16,Приходная накладная 417 от 16.01.2019 15:47:03,0,0,1.0,0.0,1.0,0
4,Склад Оборудования в СПб,833327,2019-01-16,Приходная накладная 417 от 16.01.2019 15:47:03,0,0,1.0,0.0,1.0,0


Сохраним результат, откинув лишние столбцы.

In [36]:
data[
    [
        "warehouse_id",
        "unit_id",
        "date",
        "income",
        "outcome",
        "end_balance",
        "not_valuable_send",
    ]
].to_csv("turnover.csv", index=False)

In [37]:
id = 643342627

data.query("unit_id == @id")

,warehouse_id,unit_id,date,document,sender,recipient,income,outcome,end_balance,not_valuable_send
139788,Склад Уткина Заводь (оборудование),643342627,2024-04-01,Перемещение оборудования в рознице 1 023 917 о...,Склад Уткина Заводь (оборудование),0,0.0,100.0,20700.0,0
139789,Склад Уткина Заводь (оборудование),643342627,2023-06-30,Перемещение оборудования в рознице 900 269 от ...,Склад Оборудования в СПб,Склад Уткина Заводь (оборудование),20800.0,0.0,20800.0,1
139790,Склад Оборудования в СПб,643342627,2023-06-30,Перемещение оборудования в рознице 900 269 от ...,Склад Оборудования в СПб,Склад Уткина Заводь (оборудование),0.0,20800.0,0.0,1
139791,Склад Оборудования в СПб,643342627,2019-08-30,Приходная накладная 8 359 от 30.08.2019 15:47:02,0,0,20800.0,0.0,20800.0,0
